# 主题：京东用户购买意向预测

## 故事背景

京东作为中国最大的自营式电商，在保持高速发展的同时，沉淀了数亿的忠实用户，积累了海量的真实数据。如何从历史数据中找出规律，去预测用户未来的购买需求，让最合适的商品遇见最需要的人，是大数据应用在精准营销中的关键问题，也是所有电商平台在做智能化升级时所需要的核心技术。 以京东商城真实的用户、商品和行为数据（脱敏后）为基础，通过数据挖掘的技术和机器学习的算法，构建用户购买商品的预测模型，输出高潜用户和目标商品的匹配结果，为精准营销提供高质量的目标群体。

目标：使用京东多个品类下商品的历史销售数据，构建算法模型，预测用户在未来5天内，对某个目标品类下商品的购买意向。

## 数据集 

这里涉及到的数据集是京东最新的数据集：

JData_User.csv 用户数据集 105,321个用户

JData_Comment.csv 商品评论 558,552条记录

JData_Product.csv 预测商品集合 24,187条记录

JData_Action_201602.csv 2月份行为交互记录 11,485,424条记录

JData_Action_201603.csv 3月份行为交互记录 25,916,378条记录

JData_Action_201604.csv 4月份行为交互记录 13,199,934条记录

## 数据挖掘流程

(一）、数据清洗

1. 数据集完整性验证
2. 数据集中是否存在缺失值
3. 数据集中各特征数值应该如何处理
4. 哪些数据是我们想要的，哪些是可以过滤掉的
5. 将有价值数据信息做成新的数据源
6. 去除无行为交互的商品和用户
7. 去掉浏览量很大而购买量很少的用户(惰性用户或爬虫用户)

（二）、数据理解与分析

1. 掌握各个特征的含义
2. 观察数据有哪些特点，是否可利用来建模
3. 可视化展示便于分析
4. 用户的购买意向是否随着时间等因素变化

（三）、特征提取

1. 基于清洗后的数据集哪些特征是有价值
2. 分别对用户与商品以及其之间构成的行为进行特征提取
3. 行为因素中哪些是核心？如何提取？
4. 瞬时行为特征or累计行为特征？

（四）、模型建立

1. 使用机器学习算法进行预测
2. 参数设置与调节
3. 数据集切分？

## 数据集验证

### 首先检查JData_User中的用户和JData_Action中的用户是否一致

保证行为数据中的所产生的行为均由用户数据中的用户产生（但是可能存在用户在行为数据中无行为）

思路：利用pd.Merge根据sku连接两个DataFrame, 观察数据是否减少:

In [1]:
import pandas as pd
df1 = pd.DataFrame({'sku':['a','b','c','d'],'data':[1,1,2,3]})
df2 = pd.DataFrame({'sku':['a','b','f']})
df3 = pd.DataFrame({'sku':['a','b','d']})
df4 = pd.DataFrame({'sku':['a','b','c','d']})
display(pd.merge(df1,df2))
display(pd.merge(df1,df3))
display(pd.merge(df1,df4)) # 索引一致未减少

,sku,data
0,a,1
1,b,1


,sku,data
0,a,1
1,b,1
2,d,3


,sku,data
0,a,1
1,b,1
2,c,2
3,d,3


### 查看数据

In [2]:
df_user = pd.read_csv('data/JData_User.csv')
display(df_user.head())
df_month3 = pd.read_csv('data/JData_Action_201603.csv')
df_month3.head()

,user_id,age,sex,user_lv_cd,user_reg_tm
0,200001,6.0,2.0,5,2016-01-26
1,200002,-1.0,0.0,1,2016-01-26
2,200003,4.0,1.0,4,2016-01-26
3,200004,-1.0,2.0,1,2016-01-26
4,200005,2.0,0.0,4,2016-01-26


,user_id,sku_id,time,model_id,type,cate,brand
0,280567,167208,2016-02-29 23:59:01,0.0,6,4,519
1,270248,35533,2016-02-29 23:59:02,111.0,6,4,306
2,203360,78694,2016-02-29 23:59:02,NaN,1,8,244
3,252369,90402,2016-02-29 23:59:03,0.0,6,7,38
4,279590,154208,2016-02-29 23:59:03,0.0,6,5,570


垃圾回收

In [3]:
import gc
del df_user
del df_month3
gc.collect()

0

利用pd.Merge根据user_id连接数据, 判断数据是否减少:

In [4]:
def user_action_id_check():
    df_user = pd.read_csv('data/JData_User.csv')
    df_user = df_user.loc[:,'user_id'].to_frame()
    df_month2 = pd.read_csv('data/JData_Action_201602.csv')
    print ('Is action of Feb. from User file? ', len(df_month2) == len(pd.merge(df_user,df_month2)))
    df_month3 = pd.read_csv('data/JData_Action_201603.csv')
    print ('Is action of Mar. from User file? ', len(df_month3) == len(pd.merge(df_user,df_month3)))
    df_month4 = pd.read_csv('data/JData_Action_201604.csv')
    print ('Is action of Apr. from User file? ', len(df_month4) == len(pd.merge(df_user,df_month4)))
    del df_user,df_month2,df_month3,df_month4
    gc.collect()
user_action_id_check()

Is action of Feb. from User file?  True
Is action of Mar. from User file?  True
Is action of Apr. from User file?  True


结论： User数据集中的用户和交互行为数据集中的用户完全一致

根据merge前后的数据量比对，能保证Action中的用户ID是User中的ID的子集

## 检查是否有重复记录

除去各个数据文件中完全重复的记录,可能解释是重复数据是有意义的，比如用户同时购买多件商品，同时添加多个数量的商品到购物车等...

In [6]:
def deduplicate(filepath, filename, newpath):
    df_file = pd.read_csv(filepath)       
    before = df_file.shape[0]
    df_file.drop_duplicates(inplace=True)
    after = df_file.shape[0]
    n_dup = before-after
    if n_dup != 0:
        print ('No. of duplicate records for ' + filename + ' is: ' + str(n_dup))
        df_file.to_csv(newpath, index=None)
    else:
        print ('no duplicate records in ' + filename)
    del df_file
    gc.collect()

## 检查每个数据文件重复数据

In [7]:
deduplicate('data/JData_Action_201602.csv', 'Feb. action', 'data/JData_Action_201602_dedup.csv')
deduplicate('data/JData_Action_201603.csv', 'Mar. action', 'data/JData_Action_201603_dedup.csv')
deduplicate('data/JData_Action_201604.csv', 'Apr. action', 'data/JData_Action_201604_dedup.csv')
deduplicate('data/JData_Comment.csv', 'Comment', 'data/JData_Comment_dedup.csv')
deduplicate('data/JData_Product.csv', 'Product', 'data/JData_Product_dedup.csv')
deduplicate('data/JData_User.csv', 'User', 'data/JData_User_dedup.csv')

No. of duplicate records for Feb. action is: 2756093
No. of duplicate records for Mar. action is: 7085037
No. of duplicate records for Apr. action is: 3672710
no duplicate records in Comment
no duplicate records in Product
no duplicate records in User


重复数据分析：

In [8]:
df_month3 = pd.read_csv('data/JData_Action_201603.csv')
IsDuplicated = df_month3.duplicated() 
df_d=df_month3[IsDuplicated]
# 发现重复数据大多数都是由于浏览（1），或者点击(6)产生
display(df_d.groupby('type').count())
del df_month3,df_d
gc.collect()

,user_id,sku_id,time,model_id,cate,brand
type,,,,,,
1,4659616,4659616,4659616,0,4659616,4659616
2,1406,1406,1406,0,1406,1406
3,2360,2360,2360,0,2360,2360
4,39,39,39,0,39,39
5,4475,4475,4475,0,4475,4475
6,2417141,2417141,2417141,2355765,2417141,2417141


0

# 构建user_table

为了能够进行上述清洗,在此首先构造了简单的用户(user)行为特征,对应于表user_table

user_table特征包括:
user_id(用户id),age(年龄),sex(性别),
user_lv_cd(用户级别),browse_num(浏览数),
addcart_num(加购数),delcart_num(删购数),
buy_num(购买数),favor_num(收藏数),
click_num(点击数),buy_addcart_ratio(购买加购转化率),
buy_browse_ratio(购买浏览转化率),
buy_click_ratio(购买点击转化率),
buy_favor_ratio(购买收藏转化率)

In [9]:
#定义文件名
ACTION_201602_FILE = "data/JData_Action_201602.csv"
ACTION_201603_FILE = "data/JData_Action_201603.csv"
ACTION_201604_FILE = "data/JData_Action_201604.csv"
COMMENT_FILE = "data/JData_Comment.csv"
PRODUCT_FILE = "data/JData_Product.csv"
USER_FILE = "data/JData_User.csv"
USER_TABLE_FILE = "data/User_table.csv"

## 定义函数统计用户操作频次

In [10]:
# 导入相关包
import pandas as pd
import numpy as np
from collections import Counter
# 功能函数: 对每一个user分组的数据进行统计
def add_type_count(group):
    behavior_type = group.type.astype(int)
    # 用户行为类别
    type_cnt = Counter(behavior_type)
    
    # 1: 浏览 2: 加购 3: 删除
    # 4: 购买 5: 收藏 6: 点击
    group['browse_num'] = type_cnt[1]
    group['addcart_num'] = type_cnt[2]
    group['delcart_num'] = type_cnt[3]
    group['buy_num'] = type_cnt[4]
    group['favor_num'] = type_cnt[5]
    group['click_num'] = type_cnt[6]

    return group[['user_id', 'browse_num', 'addcart_num',
                  'delcart_num', 'buy_num', 'favor_num',
                  'click_num']]

## 用户行为数据分块读取统计

由于用户行为数据量较大,一次性读入可能造成内存错误(Memory Error),因而使用pandas的分块(chunk)读取.

In [11]:
#对action数据进行统计
#根据自己调节chunk_size大小
def get_from_action_data(fname, chunk_size=50000):
    reader = pd.read_csv(fname, header=0, iterator=True)
    chunks = []
    loop = True
    while loop:
        try:
            # 只读取user_id和type两个字段
            chunk = reader.get_chunk(chunk_size)[["user_id", "type"]]
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped")
    # 将块拼接为pandas dataframe格式
    df_ac = pd.concat(chunks, ignore_index=True)
    # 按user_id分组，对每一组进行统计，as_index 表示无索引形式返回数据
    df_ac = df_ac.groupby(['user_id'], as_index=False).apply(add_type_count)
    # 将重复的行丢弃
    df_ac = df_ac.drop_duplicates('user_id')
    return df_ac

## 2月用户数据处理查看

In [12]:
df_ac = get_from_action_data(fname = ACTION_201602_FILE, chunk_size=50000)
display(df_ac.head(10))
del df_ac
gc.collect()

Iteration is stopped


,user_id,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num
0,266079,108,10,6,1,0,270
2,200719,151,12,6,5,7,233
4,263587,49,0,0,0,0,0
5,296130,71,0,0,0,0,181
8,217892,372,0,0,0,1,548
9,283139,29,9,5,3,0,4
11,243907,54,0,1,0,0,100
13,209278,47,2,0,0,0,89
20,217567,5,0,0,0,2,10
22,207684,142,10,10,1,0,217


0

## 定义函数聚合全部数据

In [13]:
# 将各个action数据的统计量进行聚合
def merge_action_data():
    df_ac = []
    df_ac.append(get_from_action_data(fname=ACTION_201602_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201603_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201604_FILE))

    df_ac = pd.concat(df_ac, ignore_index=True)
    # 用户在不同action表中统计量求和
    df_ac = df_ac.groupby(['user_id'], as_index=False).sum()
    
    #　构造转化率字段
    df_ac['buy_addcart_ratio'] = df_ac['buy_num'] / df_ac['addcart_num']
    df_ac['buy_browse_ratio'] = df_ac['buy_num'] / df_ac['browse_num']
    df_ac['buy_click_ratio'] = df_ac['buy_num'] / df_ac['click_num']
    df_ac['buy_favor_ratio'] = df_ac['buy_num'] / df_ac['favor_num']
    
    # 将大于１的转化率字段置为１(100%)
    print((df_ac['buy_addcart_ratio'] > 1.).sum())
    df_ac.loc[df_ac['buy_addcart_ratio'] > 1., 'buy_addcart_ratio'] = 1.
    df_ac.loc[df_ac['buy_browse_ratio'] > 1., 'buy_browse_ratio'] = 1.
    df_ac.loc[df_ac['buy_click_ratio'] > 1., 'buy_click_ratio'] = 1.
    df_ac.loc[df_ac['buy_favor_ratio'] > 1., 'buy_favor_ratio'] = 1.

    return df_ac

## 聚合全部数据

In [14]:
user_behavior = merge_action_data()
user_behavior.head()

Iteration is stopped
Iteration is stopped
Iteration is stopped
1430


,user_id,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
0,200001,212,22,13,1,0,414,0.045455,0.004717,0.002415,1.0
1,200002,238,1,0,0,0,484,0.000000,0.000000,0.000000,NaN
2,200003,221,4,1,0,1,420,0.000000,0.000000,0.000000,0.0
3,200004,52,0,0,0,0,61,NaN,0.000000,0.000000,NaN
4,200005,106,2,3,1,2,161,0.500000,0.009434,0.006211,0.5


从JData_User表中抽取需要的字段

In [15]:
#从JData_User表中抽取需要的字段
def get_from_jdata_user():
    df_usr = pd.read_csv(USER_FILE, header=0)
    df_usr = df_usr[["user_id", "age", "sex", "user_lv_cd"]]
    return df_usr
user_base = get_from_jdata_user()
user_base.head()

,user_id,age,sex,user_lv_cd
0,200001,6.0,2.0,5
1,200002,-1.0,0.0,1
2,200003,4.0,1.0,4
3,200004,-1.0,2.0,1
4,200005,2.0,0.0,4


## 数据存入

In [16]:
# 连接成一张表，类似于SQL的左连接(left join)
user_table = pd.merge(user_base, user_behavior, on=['user_id'], how='left')

# 保存为user_table.csv
user_table.to_csv(USER_TABLE_FILE, index=False)

display(user_table.head(10))
del user_table,user_behavior,user_base
gc.collect()

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
0,200001,6.0,2.0,5,212.0,22.0,13.0,1.0,0.0,414.0,0.045455,0.004717,0.002415,1.0
1,200002,-1.0,0.0,1,238.0,1.0,0.0,0.0,0.0,484.0,0.000000,0.000000,0.000000,NaN
2,200003,4.0,1.0,4,221.0,4.0,1.0,0.0,1.0,420.0,0.000000,0.000000,0.000000,0.0
3,200004,-1.0,2.0,1,52.0,0.0,0.0,0.0,0.0,61.0,NaN,0.000000,0.000000,NaN
4,200005,2.0,0.0,4,106.0,2.0,3.0,1.0,2.0,161.0,0.500000,0.009434,0.006211,0.5
5,200006,4.0,2.0,2,182.0,0.0,0.0,0.0,4.0,183.0,NaN,0.000000,0.000000,0.0
6,200007,4.0,2.0,3,312.0,4.0,4.0,0.0,0.0,472.0,0.000000,0.000000,0.000000,NaN
7,200008,-1.0,2.0,3,102.0,0.0,0.0,0.0,0.0,209.0,NaN,0.000000,0.000000,NaN
8,200009,4.0,2.0,2,11.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,NaN,NaN
9,200010,4.0,2.0,3,34.0,4.0,1.0,0.0,0.0,51.0,0.000000,0.000000,0.000000,NaN


0

# 数据清洗

## 用户整体性查看

In [17]:
import pandas as pd
df_user = pd.read_csv('data/User_table.csv',header=0)
pd.options.display.float_format = '{:,.3f}'.format  #输出格式设置，保留三位小数
df_user.describe()

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
count,"105,321.000","105,318.000","105,318.000","105,321.000","105,180.000","105,180.000","105,180.000","105,180.000","105,180.000","105,180.000","72,129.000","105,172.000","103,197.000","45,986.000"
mean,"252,661.000",2.773,1.113,3.850,180.466,5.471,2.434,0.459,1.045,291.222,0.147,0.005,0.009,0.552
std,"30,403.698",1.672,0.956,1.072,273.437,10.618,5.600,1.048,3.442,460.031,0.270,0.022,0.074,0.473
min,"200,001.000",-1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,"226,331.000",3.000,0.000,3.000,40.000,0.000,0.000,0.000,0.000,59.000,0.000,0.000,0.000,0.000
50%,"252,661.000",3.000,2.000,4.000,94.000,2.000,0.000,0.000,0.000,148.000,0.000,0.000,0.000,1.000
75%,"278,991.000",4.000,2.000,5.000,212.000,6.000,3.000,1.000,0.000,342.000,0.167,0.002,0.001,1.000
max,"305,321.000",6.000,2.000,5.000,"7,605.000",369.000,231.000,50.000,99.000,"15,302.000",1.000,1.000,1.000,1.000


由上述统计信息发现： 第一行中根据User_id统计发现有105321个用户，发现有3个用户没有age,sex字段，而且根据浏览、加购、删购、购买等记录却只有105180条记录，说明存在用户无任何交互记录，因此可以删除上述用户。

## 删除没有age,sex字段的用户

In [18]:
df_user[df_user['age'].isnull()]

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
34072,234073,NaN,NaN,1,32.000,6.000,4.000,1.000,0.000,41.000,0.167,0.031,0.024,1.000
38905,238906,NaN,NaN,1,171.000,3.000,2.000,2.000,3.000,464.000,0.667,0.012,0.004,0.667
67704,267705,NaN,NaN,1,342.000,18.000,8.000,0.000,0.000,743.000,0.000,0.000,0.000,NaN


In [19]:
delete_list = df_user[df_user['age'].isnull()].index
df_user.drop(delete_list,axis=0,inplace=True)
df_user.shape

(105318, 14)

## 删除无交互记录的用户

In [20]:
#删除无交互记录的用户
cond = (df_user['browse_num'].isnull()) & (df_user['addcart_num'].isnull()) & (df_user['delcart_num'].isnull()) & (df_user['buy_num'].isnull()) & (df_user['favor_num'].isnull()) & (df_user['click_num'].isnull())
df_naction = df_user[cond]
display(df_naction.head())
df_user.drop(df_naction.index,axis=0,inplace=True)
df_user.shape

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
1692,201693,-1.000,2.000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936,201937,3.000,2.000,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2273,202274,-1.000,2.000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2867,202868,3.000,2.000,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3530,203531,3.000,0.000,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(105177, 14)

## 统计并删除无购买记录的用户

In [21]:
#统计无购买记录的用户
df_bzero = df_user[df_user['buy_num']==0]
#输出购买数为0的总记录数
print(len(df_bzero))
#删除无购买记录的用户
df_user = df_user[df_user['buy_num']!=0]
df_user.describe()

75694


,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
count,"29,483.000","29,483.000","29,483.000","29,483.000","29,483.000","29,483.000","29,483.000","29,483.000","29,483.000","29,483.000","29,483.000","29,483.000","29,483.000","29,483.000"
mean,"250,746.445",2.914,1.025,4.272,302.488,10.525,4.673,1.637,1.677,486.653,0.360,0.018,0.030,0.862
std,"29,979.676",1.490,0.959,0.808,391.535,14.301,7.568,1.412,4.584,658.671,0.320,0.038,0.136,0.287
min,"200,001.000",-1.000,0.000,2.000,1.000,0.000,0.000,1.000,0.000,0.000,0.004,0.000,0.000,0.010
25%,"225,058.500",3.000,0.000,4.000,76.000,3.000,0.000,1.000,0.000,116.000,0.118,0.004,0.002,1.000
50%,"249,144.000",3.000,1.000,4.000,178.000,6.000,2.000,1.000,0.000,282.000,0.250,0.008,0.005,1.000
75%,"276,252.500",4.000,2.000,5.000,381.000,13.000,6.000,2.000,1.000,604.000,0.500,0.018,0.012,1.000
max,"305,318.000",6.000,2.000,5.000,"7,605.000",288.000,178.000,50.000,96.000,"15,302.000",1.000,1.000,1.000,1.000


## 删除爬虫及惰性用户

由上表所知，浏览购买转换比和点击购买转换比均值为0.018,0.030，因此这里认为浏览购买转换比和点击购买转换比小于0.0005的用户为惰性用户

In [24]:
bindex = df_user[df_user['buy_browse_ratio']<0.0005].index
print(len(bindex))
df_user.drop(bindex,axis=0,inplace=True)

0


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [25]:
cindex = df_user[df_user['buy_click_ratio']<0.0005].index
print(len(cindex))
df_user.drop(cindex,axis=0,inplace=True)

323


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## 数据存入

In [26]:
df_user.to_csv('./data/User_table_cleaned.csv',index=False)
df_user.describe()

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
count,"29,070.000","29,070.000","29,070.000","29,070.000","29,070.000","29,070.000","29,070.000","29,070.000","29,070.000","29,070.000","29,070.000","29,070.000","29,070.000","29,070.000"
mean,"250,767.099",2.910,1.028,4.268,280.260,10.145,4.457,1.644,1.589,447.113,0.364,0.019,0.031,0.866
std,"29,998.870",1.492,0.959,0.809,325.129,13.443,6.998,1.420,4.294,530.994,0.320,0.038,0.137,0.282
min,"200,001.000",-1.000,0.000,2.000,1.000,0.000,0.000,1.000,0.000,0.000,0.004,0.001,0.001,0.018
25%,"225,036.000",3.000,0.000,4.000,75.000,3.000,0.000,1.000,0.000,114.000,0.125,0.004,0.002,1.000
50%,"249,200.500",3.000,1.000,4.000,174.000,6.000,2.000,1.000,0.000,275.000,0.250,0.008,0.005,1.000
75%,"276,284.000",4.000,2.000,5.000,366.000,13.000,6.000,2.000,1.000,585.000,0.500,0.018,0.012,1.000
max,"305,318.000",6.000,2.000,5.000,"5,007.000",288.000,158.000,50.000,69.000,"8,156.000",1.000,1.000,1.000,1.000


最后这29070个用户为最终清洗后的用户数据集！

In [27]:
del df_user
gc.collect()

1993